---

# 4. Analysis of Context-free Languages
**[Emil Sekerinski](http://www.cas.mcmaster.ca/~emil/), McMaster University, January 2019**

---

Context-free languages can contain nested structures, which cannot be recognized by finite state machines, e.g.

    S → a S c | b

Context-free languages can be recognized by _pushdown automata_ that operate on a stack: transitions can push on and pop from the stack. The size of the stack is not bounded. Several equivalent definitions of pushdown automata exist; we use the simplest one for our purposes.

A pushdown automaton `P = (T, S, R, s₀)` is specified by

- a finite set `T` of _input symbols_,
- a finite set `S` of _stack symbols_,
- a finite set `R` of _transitions_,
- an _initial stack symbol_ `s₀ ∈ S ∪ {ε}`,

where `T` is the _vocabulary_ and each transition is a triple with a sequence `σ ∈ S*`, a symbol `t ∈ T`, and a sequence `σ' ∈ S*`, written:

	  σ t → σ'

The pushdown automaton starts with just `s₀` on the stack. A transition `σ t → σ'` can be taken if the top of the stack is `σ` and the next input symbol is `t`: the transition will pop `σ` from the stack and push `σ'` on the stack. If there are no more input symbols when the stack is empty, the input string is accepted, otherwise rejected.

For example, the automaton accepting the language `L₀ = {aⁿbⁿ | n ≥ 0}` is `P₀ = (T, S, R, s₀)` where `T = {a, b}`, `S = {.}`, `s₀ = {ε}`, and the transitions are:

  `ε a → .` <span style="float:right">(1)</span>  
  `. b → ε` <span style="float:right">(2)</span>

The input `aabb` is accepted by `P₀`:

| transition | stack | input  |
|:-----------|:------|:-------|
| ` `        | `ε`   | `aabb` |
| (1)        | `.`   | `abb`  |
| (1)        | `..`  | `bb`   |
| (2)        | `.`   | `b`    |
| (2)        | `ε`   | `ε`    |

For every finite state machine an equivalent pushdown automaton can be constructed. For example, the finite state machine accepting `E₁ = ab|ac` is `A₁ = (T, Q, R, q₀, F)` with `T = {a, b, c}`, `Q = {0, 1, 2, 3, 4}`, `q₀ = 0`, `F = {3, 4}`, and transitions `R`:

    0 a → 1
    0 a → 2
    1 b → 3
    2 c → 4

The equivalent pushdown automaton `P₁ = (T, S, R, s₀)` has the same vocabulary `T`, has `S = {0, 1}`, `s₀ = 0`, and has productions `R`:

  `0 a → 1` <span style="float:right">(1)</span>  
  `0 a → 2` <span style="float:right">(2)</span>  
  `1 b → ε` <span style="float:right">(3)</span>  
  `2 c → ε` <span style="float:right">(4)</span>

That is, the stack is initialized with the initial state of `A₁`, the transitions of `A₁` and `P₁` are the same, except that transitions in `A₁` to final states pop the state from the stack in `P₁`.

**Question.** What are the steps to accept `ab` with `P₁`?

| transition | stack | input  |
|:-----------|:------|:-------|
| ` `        | `0`   | `ab`   |
| (1)        | `1`   | `b`    |
| (3)        | ` `   | ` `    |

For every context-free grammar, an equivalent pushdown automaton can be constructed and vice versa.

As with finite state machines, pushdown automata can be deterministic or nondeterministic. Unlike with finite state machines, it is not possible in general to make a pushdown automaton deterministic and running in linear time. The best one can achieve in general is to accept in approximately `n³` time, where `n` is the length of the input.

For accepting in linear time, restrictions on the languages have to be imposed and therefore on the grammars generating these languages. There are different ways of constructing a pushdown automaton given a grammar, each with different restrictions on the grammars. Since our ultimate goal is to determine the meaning of a sentence by _parsing_, i.e. constructing a parse tree and not just to accept it, we have to be careful with modification of the grammar to suit the construction of the pushdown automaton.

_Top-down parsing_ starts to build the parse tree with the start symbol as the goal, which is split into subgoals for each non-terminal according the the grammar rules, until the terminals match the input.

Consider parsing the sentence `x * (y + z)` with grammar `G₂`:

    E → T | E + T
    T → F | T * F
    F → id | ( E )

The equivalent top-down pushdown automaton `P₂ = (T, S, R, s₀)` has the same vocabulary `T = {+, *, id, (, )}`, has stack symbols `S = {E, T, F, +, *, id, (, )}`, `s₀ = E`, and has transitions `R`:

    E ε → T
    E ε → E + T
    T ε → F
    T ε → T * F
    F ε → id
    F ε → ( E )
    id id → ε
    + + → ε
    * * → ε
    ( ( → ε
    ) ) → ε

A top-down parser takes two kinds of steps, M (_match_) and P (_produce_, _expand_), each applying one of the transitions:

| step  | stack     | input         |
|:------|:----------|:--------------|
| ` `   | `E`       | `x * (y + z)` |
| P     | `T`       | `x * (y + z)` |
| P     | `T * F`   | `x * (y + z)` |
| P     | `F * F`   | `x * (y + z)` |
| P     | `id * F`  | `x * (y + z)` |
| M     | `* F`     | `* (y + z)`   |
| M     | `F`       | `(y + z)`     |
| P     | `(E)`     | `(y + z)`     |
| M     | `E)`      | `y + z)`      |
| P     | `E + T)`  | `y + z)`      |
| P     | `T + T)`  | `y + z)`      |
| P     | `F + T)`  | `y + z)`      |
| P     | `id + T)` | `y + z)`      |
| M     | `+ T)`    | `+ z)`        |
| M     | `T)`      | `z)`          |
| P     | `F)`      | `z)`          |
| P     | `id)`     | `z)`          |
| M     | `)`       | `)`           |
| M     | ` `       | ` `           |

Because of the direct correspondence between a context-free grammar and its pushdown automaton, we will omit from now the explicit definition of the automaton.

_Bottom-up parsing_ builds the parse tree successively bottom-up by consecutively processing  input symbols; it ends when the start symbol of the grammar is reached.

Consider parsing the sentence `x * (y + z)` with the grammar:

    E → T | E + T
    T → F | T * F
    F → id | ( E )

The equivalent bottom-up pushdown automaton `P₂' = (T, S, R, s₀)` has the same vocabulary `T = {+, *, id, (, )}`, has stack symbols `S = {E, T, F, +, *, id, (, )}`, `s₀ = E`, and has transitions `R`:

    T ε → E
    E + T ε → E
    F ε → T
    T * F ε → T
    id ε → F
    ( E ) ε → F
    ε id → id
    ε + → +
    ε * → *
    ε ( → (
    ε ) → )

A bottom-up parser takes two kinds of steps, S (shift) and R (reduce), each applying one of the transitions:

| step   | stack        | input         |
|:-------|:-------------|:--------------|
| ` `    | ` `          | `x * (y + z)` |
| S      | `x`          | `* (y + z)`   |
| R      | `F`          | `* (y + z)`   |
| R      | `T`          | `* (y + z)`   |
| S      | `T *`        | `(y + z)`     |
| S      | `T * (`      | `y + z)`      |
| S      | `T * (y`     | `+ z)`        |
| R      | `T * (F`     | `+ z)`        |
| R      | `T * (T`     | `+ z)`        |
| R      | `T * (E`     | `+ z)`        |
| S      | `T * (E +`   | `z)`          |
| S      | `T * (E + z` | `)`           |
| R      | `T * (E + F` | `)`           |
| R      | `T * (E + T` | `)`           |
| R      | `T * (E`     | `)`           |
| S      | `T * (E)`    | ` `           |
| R      | `T * F`      | ` `           |
| R      | `T`          | ` `           |
| R      | `E`          | ` `           |

Bottom-up parsing proceeds without a specific goal; the parse tree grows from bottom to top; the input is accepted if is reduced to the start symbol by two kinds to step:

- Shift steps shift the next input symbol on the stack.
- Reduce steps reduce a sequence of symbols on the stack according to a production.

In pushdown automata, an input is accepted if the stack is empty, which can be achieved by adding one more transition, `E ε → ε`.

We continue with top-down parsing, which is also called _predictive parsing_ since at each P step we have to predict which production to expand. The key issue is to select the right production.

Consider grammar `G₃`:

	S → A | B
    A → x A | y
    B → x B | z

Unfortunately, when parsing `xxxz` we may get stuck:

| step | stack | input  |
|:-----|:------|:-------|
| ` `  | `S`   | `xxxz` |
| P    | `A`   | `xxxz` |
| P    | `xA`  | `xxxz` |
| M    | `A`   | `xxz`  |
| P    | `xA`  | `xxz`  |
| M    | `A`   | `xz`   |
| P    | `xA`  | `xz`   |
| M    | `A`   | `z`    |

In this situation, we would need to backtrack and revise the first P step. However, with this grammar there is no limit on the number of steps that may have to be undone.

As our goal is deterministic parsing, in case several alternative productions can be expanded, we require that the parser selects the right one with _one symbol lookahead_. The required restrictions on the grammar are expressed in terms of the _first_ and _follow sets_.

The set `first(ω)` is the set of all terminals that can appear in the first position of sentences derived from `ω`:

    first(ω) = {t ∈ T | ω ⇒* t ν, for some ν ∈ V*}
	
The set `follow(ω)` is the set of all terminal symbols that may follow `ω` in any sentence:

    follow(ω) = {t ∈ T | S ⇒* μ ω t ν, for some μ, ν ∈ V*}

Two conditions are required to ensure that a deterministic, one symbol lookahead top-down parser can be constructed.

**Condition 1.** If `A` is defined by the production

    A → χ₁ | χ₂ | …

then the initial symbols of all sentences that can be derived from all `χᵢ` must be distinct, i.e.:

  `first(χᵢ) ∩ first(χⱼ) = {}`  for all  `i ≠ j`

**Question.** What are `first(A)`, `first(B)`, `first(S)` in `G₃`?

_Answer._
- `first(A) = {x, y}`
- `first(B) = {x, z}`
- `first(S) = {x, y, z}`

Consider grammar `G₄`:

	S → A x
    A → x | ε

When parsing sentence `x`, we may again get stuck:

| step | stack | input |
|:-----|:------|:------|
| ` `  | `S`   | `x`   |
| P    | `Ax`  | `x`   |
| P    | `xx`  | `x`   |
| M    | `x`   |       |

**Condition 2.** For every nonterminal `A` from which the empty sequence can be derived, the set of initial symbols must be disjoint from the set of symbols that may follow any sequence generated from `A`:

  `first(A) ∩ follow(A) = {}`  for all `A` such that `A ⇒* ε`

**Question.** What are `first(A)` and `follow(A)` in `G₄` and from which nonterminals can `ε` be derived?

_Answer._ Only from `A` can `ε` be derived, `A ⇒* ε`, and `first(A) = {x} = follow(A)`; hence Condition 2 is violated.

The appeal of top-down parsing is that an acceptor can be directly expressed in a programming language with mutually recursive procedures by a parsing technique known as _recursive descent_. It assumes that the grammar is in EBNF.

For each nonterminal `B` defined by `B → E`, a procedure with the same name which parses sentences that can be derived from `E` is constructed,

	B → E			procedure B() pr(E)

where `pr(E)` is the parser recognizing `E`. The whole parser becomes a set of mutually recursive procedures.

Assume that procedure `next()` reads and assigns the next input symbol to global variable `sym`. The rules for constructing `pr(E)` for recognizing expression `E` are:

| `E`             | `pr(E)`                             |
|:----------------|:------------------------------------|
| `"a"`           | `if sym = "a" then next else error` |
| `B`             | `B()`                               |
| `(E₁)`          | `pr(E₁)`                            |
| `[E₁]`          | `if sym ∈ first(E₁) then pr(E₁)`    |
| `{E₁}`          | `while sym ∈ first(E₁) do pr(E₁)`   |
| `E₁ E₂ …`       | `pr(E₁) ; pr(E₂) ; …`               |
| `E1 \| E₂ \| …` | <code>case sym of<br> first(E₁): pr(E₁)<br> first(E₂): pr(E₂)<br> …<br> otherwise error</code> |

The procedure recognizing the start symbol has to be called for recognizing a sentence of the language.

For example consider the grammar `G₅`:

	A → a A c | b

Applying above rules for `pr(E)` results in:

    procedure A()
        case sym of
            "a": next ; A ; if sym = "c" then next else error end
            "b": next
        otherwise error

Above, we have already applied a number of simplifications. Generally useful transformations are:  
` `
<div style="float:left">
<code>case sym of
  L: if sym ∈ L then S else error
  …</code></div>
<span style="float:left"><code> = </code></span>
<code>case sym of
   L: S
   …</code>
<br><br>
<div style="float:left">
<code>while sym ∈ L do
  if sym ∈ L then S else error</code></div>
<span style="float:left"><code> = </code></span>
<code>while sym ∈ L do
   S</code>
<br><br>
<div style="float:left">
<code>case sym of
  L1: S1
  L1: S1
  …
otherwise error</code></div>
<span style="float:left"><code> = </code></span>
<code>if sym = L1 then S1
  else if sym = L2 then S2
  …
  else error</code>

The implementation in Python stores the input in the global variable `src` and maintains an index to the next symbol to be read:

In [ ]:
def nxt():
    global pos, sym
    if pos < len(src): sym, pos = src[pos], pos+1
    else: sym = chr(0) # end of input symbol

def A():
    if sym == 'a':
        nxt(); A();
        if sym == 'c': nxt()
        else: raise Exception("'c' expected at " + str(pos))
    elif sym == 'b': nxt()
    else: raise Exception("'a' or 'b' expected at " + str(pos))

def parse(s):
    global src, pos;
    src, pos = s, 0; nxt(); A()
    if sym != chr(0): raise Exception("unexpected characters at " + str(pos))

parse("aabcc")